# This is a notebook to generate synthetic data of overlapping particles

In [ ]:
# import some necessary python modules 
import numpy as np
import yaml
import pandas as pd
import tqdm
from pathlib import Path

In [ ]:
# import the user defined modules 
from image_generate_modules import gen_path, image_generator, binary_to_heatmap
from utils.config_utils import load_config

## Set up the file architecture 

In [10]:
# declare necessary cirectory architecture
# ........... build the path architecture ........ 
# set current directory
cwd = Path().resolve() 
Parent = cwd.parent
# set file path architecture 
path_data = load_config(str(cwd.parent / 'configs/config.yaml'))
# path to synthetic training data 
train_data_syn = cwd.parent /path_data['Images']['FirstTrain_syn']
# path to the heatmaps (parquet and the .npy)
parquet_path = cwd.parent /path_data['Heatmaps']['Parquet_Transfer']
heatmap_data_syn = cwd.parent /path_data['Heatmaps']['FirstTrain_syn']

# Location to store the training images 
syn_img_folder = str(train_data_syn)
syn_heatmap_folder = str(heatmap_data_syn )

# location to store the 

## define some of the parameters

In [1]:
# define the frame dimensions
X_size = 128            # x dimension of the image in pixels 
Y_size = 128            # y dimension of the image in pixels 

# dataset parameters
N_images = 10           # number of images
nbParticles = 500       # The approximate number of particles 
PtoB_ratio  = 17/512    # PtoB_ratio maps the particle size in microns to the image in the pixels 
Packing_fraction = 0.5  # packing fraction of the particles 

# heatmap parameters
sigma = 0.7             # spead of the gaussian for the heatmap 

##  Create the Images of the overlapping particles 

In [ ]:
# create path locations for saveing the synthetic images
path_list = gen_path(syn_img_folder, N_images) 

# stores data of the particle postion as 2D binary mask
H_list = []

# create a bunch of images  
for i in tqdm.tqdm(range(N_images)) :
    X_, Y_, H = image_generator(path_list[i], [X_size, Y_size], nbParticles , PtoB_ratio=PtoB_ratio, packing_frac = Packing_fraction)
    # flatten the dataframe 
    H_list.append(H.flatten())

## Store the 2D particle positions as a flattened DataFrame as .parquet file 

In [16]:
# save the 2D logistic map parquet format ... serves as a metadata ....
parquet_name = 'temp.parquet' # ...  or some other naem based on use case 
H_val = pd.DataFrame()
H_val['H'] = H_list
H_val.to_parquet(str(parquet_path /'temp.parquet'))

## Create the Corresponding heatmaps

In [ ]:
# create heatmaps
# create path locations for saveing the synthetic image heatmaps 
path_list = gen_path(syn_heatmap_folder, N_images)

# run the loop for developing the synthetic image heatmaops 
for i in tqdm.tqdm(range(len(H_val))):
    
    H = H_val['H'].iloc[i]
        
    # compute and store the heatmaps     
    binary_to_heatmap(path_list[i], sigma, H, [X_size, Y_size])
